# 固定効果モデル

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrix

In [2]:
filepath = '../../../../data/processed/df_filtered_5years_exclude_too_far_islands.xlsx'
df = pd.read_excel(filepath)
df = df[df['island_id'] != 191] # この島は変化率が無限大になるので除外
df.dropna(inplace=True)

In [3]:
df['population_change_rate'] = df.groupby('island_id')['population'].pct_change()
df['log_income'] = np.log(df['income'])
df['log_area'] = np.log(df['area_km2'])
df['log_distance'] = np.log(df['distance_m'])
df = df.dropna()

In [8]:
y = df['population_change_rate']
X = dmatrix('dummy_after_bridge_opened + log_income -1', data=df, return_type='dataframe')
islands = dmatrix('C(island_id) -1', data=df, return_type='dataframe')
years = dmatrix('C(year) -1', data=df, return_type='dataframe')

X = pd.concat([X, islands, years], axis=1)

model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                              OLS Regression Results                              
Dep. Variable:     population_change_rate   R-squared:                       0.267
Model:                                OLS   Adj. R-squared:                  0.063
Method:                     Least Squares   F-statistic:                     1.307
Date:                    Tue, 29 Oct 2024   Prob (F-statistic):             0.0428
Time:                            12:55:28   Log-Likelihood:                 272.60
No. Observations:                     446   AIC:                            -349.2
Df Residuals:                         348   BIC:                             52.62
Df Model:                              97                                         
Covariance Type:                nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------